Not all cells may work. May require generating files and cases through a microphysics run first

In [37]:
# Packages
import numpy as np
import scipy.io.netcdf as S
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from scipy import interpolate
import mpl_toolkits.basemap as bm
import xarray as xr
import esmlab
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 10]
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

/glade/derecho/scratch/wchuang/tmp/ipykernel_2481/1193198533.py:14: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png', 'pdf')


In [38]:
# Some Parameters
seastx='DJF'
iseas=[11,0,1]  #DJF

# Select region
ylat=(-60,-35)
xlon=(130,165)

In [39]:
# Model data

pth='/glade/derecho/scratch/wchuang/'

cases=[
       'run13_kk2000_120month',
       # 'run13_tau_120month',
       # 'run13_1nn_optimized_120month',
       # 'run12_3nn_add_CLD_lev_FREQR_12month_proper_rerun',
      ]

cstxt=[
       'KK2000 bulk',
       # 'TAU-bin',
       # 'TAU-ML',
       # 'TAU 3nn',
      ]

tmp = [pth + s for s in cases]
# fspec = [s + '/atm/hist/' for s in tmp]
fspec = [s + '/run/' for s in tmp] # temporary until archive is set up

nr=len(cases)

In [40]:
fspec[0]
#d1= xr.open_dataset('/glade/scratch/andrew/archive/cam600_base_AE2/atm/hist/cc_cam600_base_AE2.h0.FICE.nc')

'/glade/derecho/scratch/wchuang/run13_kk2000_120month/run/'

In [41]:
#Select variable
varn='PRECT'
lgscale=True
ctxt='Large Scale'
fnprefix='PRECL'
scl=86400.*1000.  # m/s --> mm/d
vunits='mm/d'
minpr=0.1
# fsuff='.h1.'+varn+'.nc'
# fsuff2='.h1.PRECC.nc'

Read Data and Set up big arrays

In [ ]:
for r in range(nr):

    #Read Model Data for needed fields....(extract variable first)

    fn=fspec[r]+'*.cam.h1.*'
    d1= xr.open_mfdataset(fn)

    fi=d1[varn]*scl
    
#Subtract convective to get large scale if desired.     
    if lgscale:
        fn2=fspec[r]+'*.cam.h1.*'
        d2= xr.open_mfdataset(fn2)
        fi=d1[varn]*scl - d2['PRECC']*scl

    fi.attrs['units']=vunits
  
    if r==0:
        coordinates={'runs':cstxt,'time':fi.time,'lon':fi.lon,'lat':fi.lat}
        ny=len(fi.lat)
        nx=len(fi.lon)
        nt=len(fi.time)
        fiall = xr.DataArray(np.zeros((nr,nt,ny,nx)),dims=('runs','time','lat','lon'),coords=coordinates)
    fiall[r,:,:,:]=fi[:,:,:]

In [ ]:
#make some regions (lat and lon)
#   rgx=[[120,160.],[0,360.],[0,360],[300,360],[260,295],[0,360]]
#   rgy=[[-20,20],[65,80],[-65,-50],[40,60],[-30,-10],[-90,90]]

rgn=['TWP','Arctic','S.Ocean','N.Atl','S.E.Pac','Global']
ng=len(rgn)
bnd=['min','max']
cc={'region':rgn,'bounds':bnd}
rgnx = xr.DataArray(np.zeros((ng,2)),dims=('region','bounds'),coords=cc)
rgny = xr.DataArray(np.zeros((ng,2)),dims=('region','bounds'),coords=cc)
rgnx[0,:]=[120.,160.]
rgnx[1,:]=[0,360.]
rgnx[2,:]=[0,360]
rgnx[3,:]=[300,360]
rgnx[4,:]=[260,295]
rgnx[5,:]=[0,360]
rgny[0,:]=[-20,20]
rgny[1,:]=[65,80]
rgny[2,:]=[-65,-50]
rgny[3,:]=[40,60]
rgny[4,:]=[-30,-10]
rgny[5,:]=[-90,90]

In [ ]:
#exclude zero rain rates with a where function...

binedg=[0.1,1.,2.,5.,10,20,50,100,150,200,300,400,500,600.]
nb=len(binedg)-1
binctr = np.zeros(nb)
for i in range(nb):
    binctr[i]=binedg[i]+(binedg[i+1]-binedg[i])/2.
    
binctr

In [ ]:
#calculate a histogram,pdf

histall=np.zeros((nr,nb))

for r in range(nr):
    prhist=fiall[r,:,:,:].where(fiall > minpr)
    hist, bin_edges = np.histogram(prhist,bins=binedg,density=True)
    norm=hist/hist.sum()
    histall[r,:]=norm

In [ ]:
#Figure out time range here....
fiall.time
# 3 hourly data, so total is 720 days
# 365*2*8/(8*30), or 8*30 in a month....
mot=8*30+3
m=23
tr=[mot*m,mot*(m+1)]
tmp=fiall[0,:,:,:].isel(time=slice(tr[0],tr[1]))
#tmp

In [ ]:
# calculate a histogram each month for control case...

mot=8*30+3
nmo=12  #hardcode months (not great)
r=2 #hard code run (not great)

histmon=np.zeros((nmo,nb))

for m in range(nmo):
    tr=[mot*m,mot*(m+1)]
    tmp=fiall[0,:,:,:].isel(time=slice(tr[0],tr[1]))
    # tmp=fiall[2,:,:,:].isel(time=slice(tr[0],tr[1]))
    prhist=tmp.where(tmp > minpr)
    hist, bin_edges = np.histogram(prhist,bins=binedg,density=True)
    norm=hist/hist.sum()
    histmon[m,:]=norm

In [ ]:
#now loop over bins and take standard deviation....

histstd=np.zeros(nb)
for b in range(nb):
    histstd[b]=np.std(histmon[:,b])

In [ ]:
savfig=False

#Plot...

sns.set_context("talk")

ttl=ctxt+' Precipitiation Intensity'
xtl='Precip [mm/day]'
ytl='Frequency'

for r in range(nr):
    plt.plot(binctr,histall[r,:],label=cstxt[r])
#plt.fill_between(binctr,histall[2,:]-histstd,histall[2,:]+histstd,alpha=0.3)
plt.yscale('log')
plt.legend()
plt.title(ttl)
plt.xlabel(xtl)
plt.ylabel(ytl)

if savfig:
    plt.savefig('./figs/' + fnprefix + 'run13_intensity1_nncompare.pdf')